In [1]:
!git clone https://github.com/Mike030668/Spliter_Text2Movi_Kandinskiy_22.git -q

# Construct method to find points of start and end fragments movi

In [9]:
import numpy as np
import torch
from torch import nn
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAIN_DIR =  "/content/Spliter_Text2Movi_Kandinskiy_22"
import sys
sys.path.append(MAIN_DIR)

## init rand data

In [11]:
from utills.step_points import ComputeDiffPoints

In [12]:
cdp = ComputeDiffPoints()

# make set rand embeddings for tests
d_batch = 15
MAX_LEN_MOVI = 100

# sets images embeds
image_embeds = torch.randn(d_batch+1, 1, 1280)

# text embedings prompt video from Text model
text_hid_state = torch.randn((1, 77, 1280), requires_grad=True)

# unclip embedings prompt video from Prior
unclip_embed = torch.randn((1, 1, 1280), requires_grad=True)

# make set time_labels
labels = np.random.choice(np.arange(1, MAX_LEN_MOVI), d_batch+1, replace=False)

# start and end points to use for start normal and back ways
labels[0] = 0
labels[-1] = MAX_LEN_MOVI-1

print(labels)
cdp.time_labels = labels
config_norm, config_back = cdp.getpoints_train()

print(f"Time labels {labels}\n")
print('\nNorm_way')
print(f"Start base point {cdp.s_point}")
print(f"Base normal points {cdp.points}\n\n")
print('\nBack_way')
print(f"Start base back_point {cdp.back_s_point}")
print(f"Base back points {cdp.back_points}\n\n")

[ 0  1 77 49 70 44 28 76 20  2  7 62 35 56 61 99]
Time labels [ 0  1 77 49 70 44 28 76 20  2  7 62 35 56 61 99]


Norm_way
Start base point 0
Base normal points [ 1 77 49 70 44 28 76 20  2  7 62 35 56 61 99]



Back_way
Start base back_point 0
Base back points [38 43 64 37 92 97 79 23 71 55 29 50 22 98 99]




## compute normal and back points for rotation steps

In [13]:
print(f"Time labels {labels}")
print('\n\nNorm rotation way')
print(f"Base normal points {cdp.points} for start rotation steps \n")
id_img_emb_s = config_norm["id_img_emb_s"]
print(f"ID image and text start rotation points {id_img_emb_s}")
id_uclip_emb = config_norm["id_uclip_emb"]
print(f"ID uclip_emb start rotation points {id_uclip_emb}")
id_img_delta = config_norm["id_img_delta"]
print(f"ID delta rotation points {id_img_delta}")
delta = config_norm["delta"]
print(f"Delta between points {delta} normal points")


print('\n\nNorm rotation way')
print(f"Base back points {cdp.back_points} for start rotation back steps \n")
id_img_emb_s = config_back["id_img_emb_s"]
print(f"ID image and text start rotation points {id_img_emb_s} in base back points")
id_uclip_emb = config_back["id_uclip_emb"]
print(f"ID uclip_emb start rotation points {id_uclip_emb} in base back points")
id_img_delta = config_back["id_img_delta"]
print(f"ID delta rotation points {id_img_delta} in base back points")
delta = config_back["delta"]
print(f"Delta between points {delta} back points")


Time labels [ 0  1 77 49 70 44 28 76 20  2  7 62 35 56 61 99]


Norm rotation way
Base normal points [ 1 77 49 70 44 28 76 20  2  7 62 35 56 61 99] for start rotation steps 

ID image and text start rotation points [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 0, 4, 5, 7, 8, 9, 11, 0, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 0, 5, 7, 8, 9, 11, 0, 7, 8, 9, 0, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 0, 8, 9, 0, 0, 8, 0, 2, 4, 5, 7, 8, 9, 11, 12, 13, 0, 5, 7, 8, 9, 0, 2, 4, 5, 7, 8, 9, 11, 0, 2, 4, 5, 7, 8, 9, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
ID uclip_emb start rotation points [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 0, 4, 5, 7, 8, 9, 11, 0, 2, 4, 5, 7, 8, 9, 10, 11, 12, 13, 0, 5, 7, 8, 9, 11, 0, 7, 8, 9, 0, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 0, 8, 9, 0, 0, 8, 0, 2, 4, 5, 7, 8, 9, 11, 12, 13, 0, 5, 7, 8, 9, 0, 2, 4, 5, 7, 8, 9, 11, 0, 2, 4, 5, 7, 8, 9, 11, 12, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
ID delta rotation points [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,

## compute normal and back points for diff steps

It will be used for prediction from prediction of model

In [14]:
config_diff_norm, config_diff_back = cdp.getpoints_diftrain()

print(f"Time labels {labels}")
print('\n\nNorm diff way')
print(f"Base normal points {cdp.points}")
print(f"Next normal points {cdp.next_points} for diff steps \n")
id_img_emb_s = config_diff_norm["id_img_emb_s"]
print(f"ID image and text start diff points {id_img_emb_s} in base normal points")
id_uclip_emb = config_diff_norm["id_uclip_emb"]
print(f"ID uclip_emb start diff points {id_uclip_emb} in next normal points")
id_img_delta = config_diff_norm["id_img_delta"]
print(f"ID delta diff points {id_img_delta} in next normal points")
delta = config_diff_norm["delta"]
print(f"Delta between diff points {delta} in base normal points")


print('\n\nNorm rotation way')
print(f"Base back points {cdp.back_points}")
print(f"Next back points {cdp.back_next_points} for diff steps \n")
id_img_emb_s = config_diff_back["id_img_emb_s"]
print(f"ID image and text start diff points {id_img_emb_s} in base back points")
id_uclip_emb = config_diff_back["id_uclip_emb"]
print(f"ID uclip_emb start diff points {id_uclip_emb} in next back points")
id_img_delta = config_diff_back["id_img_delta"]
print(f"ID delta diff points {id_img_delta} in next back points")
delta = config_diff_back["delta"]
print(f"Delta between diff points {delta} back points")


Time labels [ 0  1 77 49 70 44 28 76 20  2  7 62 35 56 61 99]


Norm diff way
Base normal points [ 1 77 49 70 44 28 76 20  2  7 62 35 56 61 99]
Next normal points [  2  78  50  71  45  29  77  21   3   8  63  36  57  62 100] for diff steps 

ID image and text start diff points [8, 10, 8] in base normal points
ID uclip_emb start diff points [0, 13, 0] in next normal points
ID delta diff points [1, 1, 10] in next normal points
Delta between diff points [75, 15, 60] in base normal points


Norm rotation way
Base back points [38 43 64 37 92 97 79 23 71 55 29 50 22 98 99]
Next back points [ 39  44  65  38  93  98  80  24  72  56  30  51  23  99 100] for diff steps 

ID image and text start diff points [7, 0, 7, 0, 7, 13] in base back points
ID uclip_emb start diff points [12, 3, 12, 3, 12, 5] in next back points
ID delta diff points [0, 13, 13, 14, 14, 14] in next back points
Delta between diff points [15, 60, 75, 61, 76, 1] back points


# Both ways steps

In [19]:
from models.baseline import SpliterSimple
from utills.clearing import flush_memory

model = SpliterSimple(max_delta_time = 100,
                        emb_dim = 1280,
                        ways = 2
                        )

## Losses

nn.CosineEmbeddingLoss
https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html

In [20]:
clip_text_last_hidden_state = torch.randn((2, 77, 1280)) #outputs.last_hidden_state
embeddings = torch.randn((2, 1, 1280))
delta = torch.tensor([[3],[5]])
direction  = torch.tensor([[0],[1]])
model.to(DEVICE)
out_embeddings = model(text_hidden_states = clip_text_last_hidden_state.to(DEVICE),
                       prior_embeds = embeddings.to(DEVICE),
                       delta_time = delta.to(DEVICE),
                       direction = direction.to(DEVICE)
                        )
out_embeddings.shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2560x1280 and 128x64)

In [ ]:
# https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html
loss = nn.CosineEmbeddingLoss()
input1 = torch.randn(3, 1, requires_grad=True)
input2 = torch.randn(3, 5, requires_grad=True)
target = torch.ones(3)
output = loss(input1, input2, target)
output.backward()
output

tensor(-0.0026, grad_fn=<MeanBackward0>)

## apply as vector losses

In [ ]:
mse_loss = nn.MSELoss(reduction='none').to(DEVICE)
cos_loss = nn.CosineEmbeddingLoss(reduction = 'none').to(DEVICE)
target = torch.ones(1280).to(DEVICE)
loss_cos = cos_loss(embeddings.squeeze(1).T.to(DEVICE), out_embeddings.squeeze(1).T, target)
print(loss_cos)

tensor([1.8691, 0.2069, 1.8156,  ..., 0.8714, 0.1041, 1.6076], device='cuda:0',
       grad_fn=<AddBackward0>)
